In [ ]:
!pip install ipynb --upgrade

In [5]:
from ipynb.fs.full.NER import get_entities
import pandas as pd 
import datetime
import requests
from collections import Counter
import glob

ModuleNotFoundError: No module named 'ipynb.fs.full.NER'

In [ ]:
#get_entities('this GME is making me rich')

### Load csv

In [ ]:
#df_clean = pd.read_csv('data_files/wallstreetbets_21.csv', lineterminator='\n')
data = pd.read_csv('../Data-Science-Project/Data_Collection/data_files/Stocks_34.csv')
data

In [ ]:
d = pd.read_csv('./Data_Collection/data_files/Stocks_3.csv')
d

In [ ]:
old_filenames = []
old_filenames.append('Data_Collection/data_files/wallstreetbets_50.csv')

In [ ]:
old_filenames.append('Data_Collection/data_files/wallstreetbets_44.csv')

In [ ]:
# read all csv files
for file_name in glob.glob('./Data_Collection/data_files/'+'*.csv'):
    print('* '+file_name)
    if file_name not in old_filenames:
        data = pd.read_csv(file_name,lineterminator='\n')
        # Transform data
        print('   * Transforming data')
        dfs = transform_reddit_data(data)
        df_submissions = dfs[0]
        df_comments = dfs[1]
        # Persist submissions and comments 
        print('   * Persisting submissions')
        insert_submissions(df_submissions)
        print('   * Persisting comments')
        insert_comments(df_comments)
        old_filenames.append(file_name)

# TESTING

In [ ]:
data = pd.read_csv('./Data_Collection/data_files/Stocks_3.csv')
df_submissions = data.drop(['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title'],axis=1)
df_submissions['created'] = df_submissions['created'].apply(datetime_to_date)
df_submissions = df_submissions.dropna()
df_submissions.rename(columns = {'post_id':'id'}, inplace = True)
df_submissions['Organizations'] = df_submissions['text'].apply(get_entities).apply(clean_orgs)
insert_submissions(df_submissions)
df_comments = data.drop(['Unnamed: 0','post_id', 'fullname', 'title'],axis=1)
df_comments['created'] = df_comments['created'].apply(datetime_to_date)
df_comments = df_comments.dropna()
df_comments['Organizations'] = df_comments['text'].apply(get_entities).apply(clean_orgs)
insert_comments(df_comments)


----

## Transforming the data

In [ ]:
# Returns list with submissions dataframe and comments dataframe
def transform_reddit_data(data: pd.DataFrame):
        dfs = []
        # submissions
        #candidates =['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title']
        #df_submissions = data.drop([x for x in candidates if x in data.columns], axis=1)
        df_submissions = data.drop(['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title'],axis=1)
        df_submissions['created'] = df_submissions['created'].apply(datetime_to_date)
        df_submissions = df_submissions.dropna()
        df_submissions.rename(columns = {'post_id':'id'}, inplace = True)
        df_submissions['Organizations'] = df_submissions['text'].apply(get_entities).apply(clean_orgs)
        dfs.append(df_submissions)
        # comments
        #candidates =['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title']
        #df_comments = data.drop([x for x in candidates if x in data.columns], axis=1)
        df_comments = data.drop(['Unnamed: 0','post_id', 'fullname', 'title'],axis=1)
        df_comments['created'] = df_comments['created'].apply(datetime_to_date)
        df_comments = df_comments.dropna()
        df_comments['Organizations'] = df_comments['text'].apply(get_entities).apply(clean_orgs)
        dfs.append(df_comments)
        
        return dfs

In [ ]:
# Changes utc timestamp to datetime.date
def datetime_to_date(timestamp):
    return pd.to_datetime(timestamp).date()

In [ ]:
# Mentions that we are interested in
selected_orgs =  ['HKG', 'Alibaba','AMC', 'Palantir Technologies', 'PLTR', 'FORD', 'Lordstown Motors', 'RIDE', 'Virgin Galactic', 'SPCE', 'AI', 'C3.AI', 'TSLA', 'GE', 'GME', 'AAPL', 'Tesla', 'Apple', 'General Electric', 'GE', 'NOK', 'Nokia']
orgs_dict = {"Alibaba":"HKG","AMC":"AMC","Palantir Technologies":"PLTR","FORD":"FORD", "Lordstown Motors":"RIDE","Virgin Galactic":"SPCE","c3.AI":"AI","Tesla":"TSLA", "General Electric":"GE","Apple":"AAPL","GameStop":"GME","Gamestop":"GME", "Nokia":"NOK"}


Clean orgs

In [ ]:
# returning a list of mentioned tickers
def clean_orgs(organizations):
    orgs = []
    for org in organizations:
        if org in selected_orgs:
            if org in orgs_dict:
                org = orgs_dict[org]
                orgs.append(org)
            else: 
                orgs.append(org)
    for org in orgs:
        o = set(orgs)
        orgs = list(o)
    return orgs

In [ ]:
clean_orgs(['Virgin Galactic'])

## A Look at the mentions

In [ ]:
# Creates a list of mentionend entities

#data['Organizations'] = data['text'].apply(get_entities)
#orgs = data['Organizations'].to_list()
#orgs_flat = [org for sublist in orgs for org in sublist] # Pulls out entities from the nested lists in orgs => new flat list
# Print 20 most mentions ORGs
#from collections import Counter
#org_freq = Counter(orgs_flat)
#org_freq.most_common(20)                                                       

## Transform and save the data to database

In [ ]:
#submissionsransforming into lists of submissions and comments
dfs = transform_reddit_data(data)
submissions = dfs[0]
comments = dfs[1]

In [ ]:
## SUBMISSIONS subset
#print(len(submissions))
df = submissions[1:500]
insert_submissions(df)
#COMMENTS subset
#print(len(comments))
#df_c = comments[500:5500]
#
#insert_comments(df_c)

#insert_c(df_c)

In [ ]:
url = 'http://localhost:5050/'

In [ ]:
def insert_submissions(dataframe: pd.DataFrame):
    
    for index, row in dataframe.iterrows():
        #create submission nodes
        url2 = 'submission/'+row['subreddit']+'/'+row['created'].strftime('%Y-%m-%d')+'/'+row['id']
        response = requests.post(url+url2)
        if not response.status_code == 200:
            print ('submission id '+ row['id'] + ' failed insertion')
        # create mentions relationships
        for org in row['Organizations']:
            url3 = 'connection/mentions/submission/'+row['id']+'/'+ org + '/' + row['created'].strftime('%Y-%m-%d')
            response = requests.post(url+url3)
            if not response.status_code == 200:
                print ('mentions between '+ row['id'] + ' and ' + org + ' failed creation')

In [ ]:
def insert_comments(dataframe: pd.DataFrame):
    
    for index, row in dataframe.iterrows():
        
        #create comment nodes
        url2 = 'comment/'+row['subreddit']+'/'+row['created'].strftime('%m-%d-%Y')+'/'+row['comment_id']+'/'+row['parent_id']
        response = requests.post(url+url2)
        if not response.status_code == 200:
            print ('comment id '+ row['comment_id'] + ' failed insertion')
            
        # create mentions relationships
        for org in row['Organizations']:
            url3 = 'connection/mentions/comment/'+row['comment_id']+'/'+ org +'/'+ row['created'].strftime('%m-%d-%Y')
            response = requests.post(url+url3)
            if not response.status_code == 200:
                print ('mention between '+ row['comment_id'] + ' and ' + org + ' failed creation')
    
           
        # create replies relationships
        row['parent_id'] = pd.Series(row['parent_id'], dtype="string") #turning parent_id from type Series to String
        for parent_id in row['parent_id']:
            p_id = parent_id[3:]
            if parent_id[:2] == 't1':
                url_reply = 'connection/replies/comment/' + row['comment_id'] + '/' + p_id
            elif parent_id[:2] == 't3':
                url_reply = 'connection/replies/submission/' + row['comment_id'] + '/' + p_id
            response = requests.post(url+url_reply)
            if not response.status_code == 200:
                print ('replies relationship between '+ row['comment_id'] + ' and ' + p_id + ' failed creation')